# Connect to DB

In [ ]:
"""
Create a"database.ini"-file, which contains the following information. 
[postgresql]
host=...
database=...
user=...
password=...
"""
import psycopg2
from config import config

try:
    params = config()
    print("Connecting to the PostgreSQL database...")
    conn = psycopg2.connect(**params)
    
    cur = conn.cursor()
    print("Connected")
except(Exception, psycopg2.DatabaseError) as error: 
    print(error)

# Data Creation and Insertion to DB

The data for each table is first randomly created as a list of tuples. After that, the list is being inserted into the corresponding table of the DB. Then, the changes are commited to the DB to avoid errors due to dependencies. Some tables are read so they can be an input for further data generation. This procedure is repeated for all tables in the DB.

In [ ]:
import pandas as pd
from ReferenceDataGenerator import generate_MA_EMPLOYEES
from PDDataGenerator import *
from ProductionDataGenerator import *
from SalesDataGenerator import *
from DB_insert_select import *

In [ ]:
num_rows = 300

## Employee 

In [ ]:
num_employees = num_rows
employees = generate_MA_EMPLOYEES(num_employees,1)
insert_into("sppl.MA_EMPLOYEES", 5, employees, cur, conn)
emp = pd.read_sql_query("SELECT * FROM sppl.MA_EMPLOYEES", conn)

## Product Development

In [ ]:
num_experiments = num_rows
pd_experiments, cols = generate_PD_EXPERIMENTS(num_experiments, 1)
insert_into("sppl.PD_EXPERIMENTS", 24, pd_experiments, cur, conn, col = cols)
exp = pd.read_sql_query("SELECT * FROM sppl.PD_EXPERIMENTS", conn)

In [ ]:
num_ingredients = num_rows
pd_ingredients, cols = generate_PD_INGREDIENTS(num_ingredients, exp)
insert_into("sppl.PD_INGREDIENTS", 5, pd_ingredients, cur, conn, col = cols)
ing = pd.read_sql_query("SELECT * FROM sppl.PD_INGREDIENTS", conn)

In [ ]:
num_internal_ratings = num_rows
existing_internal_ratings = pd.read_sql_query("SELECT * FROM sppl.PD_INTERNAL_RATINGS", conn)
pd_internal_ratings = generate_PD_INTERNAL_RATINGS(num_rows, exp,existing_internal_ratings)
insert_into("sppl.pd_internal_ratings", 3, pd_internal_ratings, cur, conn)
pd_internal_ratings = pd.read_sql_query("SELECT * FROM sppl.PD_INTERNAL_RATINGS", conn)

In [ ]:
num_external_ratings = num_rows
existing_external_ratings = pd.read_sql_query("SELECT * FROM sppl.PD_EXTERNAL_RATING_EVENTS", conn)
pd_external_rating_events = generate_PD_EXTERNAL_RATING_EVENTS(num_external_ratings,exp, existing_external_ratings)
insert_into("sppl.PD_EXTERNAL_RATING_EVENTS", 14, pd_external_rating_events, cur, conn)

# Production

In [ ]:
#PROD_INGREDIENTS
num_ing = num_rows
prod_ingredients, cols = generate_PROD_INGREDIENTS(num_ing, 1)
insert_into("sppl.PROD_INGREDIENTS", 2, prod_ingredients, cur, conn, col = cols)
ing = pd.read_sql_query("SELECT * FROM sppl.PROD_INGREDIENTS", conn)

In [ ]:
num_rec = num_rows
prod_recipies = generate_PROD_RECIPIES(num_rec, 1, emp)
insert_into("sppl.PROD_RECIPES", 5, prod_recipies, cur, conn)
rec = pd.read_sql_query("SELECT * FROM sppl.PROD_RECIPES", conn)

In [ ]:
num_rec_instruct = num_rows*5
prod_recipies_instructions, cols = generate_PROD_RECIPIES_INSTRUCTIONS(num_rows, rec)
insert_into("sppl.PROD_RECIPES_INSTRUCTIONS", 2, prod_recipies_instructions, cur, conn, col = cols)
pri = pd.read_sql_query("SELECT * FROM sppl.PROD_RECIPES_INSTRUCTIONS", conn)

In [ ]:
num_ing_adds = num_rows
prod_ing_add = generate_PROD_RECIPES_INSTRUCTIONS_INGREDIENT_ADDS(num_rows, 1, pri, ing)
insert_into("sppl.PROD_RECIPES_INSTRUCTIONS_INGREDIENT_ADDS", 3, prod_ing_add, cur, conn)

In [ ]:
num_mash = num_rows
prod_mash = generate_PROD_RECIPES_INSTRUCTIONS_MASHINGS(num_mash, 1, pri)
insert_into("sppl.PROD_RECIPES_INSTRUCTIONS_MASHINGS", 4, prod_mash, cur, conn)

In [ ]:
num_mash_rest = num_rows
prod_mash_rest = generate_PROD_RECIPES_INSTRUCTIONS_MASHINGS_RESTS(num_mash_rest, 1, pri)
insert_into("sppl.PROD_RECIPES_INSTRUCTIONS_MASHING_RESTS", 2, prod_mash_rest, cur, conn)

In [ ]:
num_spargings = num_rows
prod_spargings = generate_PROD_RECIPES_INSTRUCTIONS_SPARGINGS(num_spargings, 1, pri)
insert_into("sppl.PROD_RECIPES_INSTRUCTIONS_SPARGINGS", 2, prod_spargings, cur, conn)

In [ ]:
num_boilings = num_rows
prod_boilings = generate_PROD_RECIPES_INSTRUCTIONS_BOILINGS(num_boilings, 1, pri)
insert_into("sppl.PROD_RECIPES_INSTRUCTIONS_BOILINGS", 2, prod_boilings, cur, conn)

Ingredient Products:

In [ ]:
num_ing_prod = num_rows
prod_ing_prod, cols = generate_PROD_INGREDIENT_PRODUCTS(num_ing_prod, 1, ing)
insert_into("sppl.PROD_INGREDIENT_PRODUCTS", 4, prod_ing_prod, cur, conn, col = cols)
ing_prod = pd.read_sql_query("SELECT * FROM sppl.PROD_INGREDIENT_PRODUCTS", conn)

Brews main table:

In [ ]:
num_brews = num_rows*7
prod_brews, cols = generate_PROD_BREWS(num_brews, 1, emp, rec)
insert_into("sppl.PROD_BREWS", 6, prod_brews, cur, conn, col = cols)
brews = pd.read_sql_query("SELECT * FROM sppl.PROD_BREWS", conn)

Brew events:

In [ ]:
num_brews_ing_add = num_rows
prod_brews_ing_add = generate_PROD_BREWS_INGREDIENT_ADDS(num_brews, 1, brews, ing_prod)
insert_into("sppl.PROD_BREWS_INGREDIENT_ADDS", 4, prod_brews_ing_add, cur, conn)

In [ ]:
num_brews_mash = num_rows
prod_brews_mash = generate_PROD_BREWS_MASHINGS(num_brews_mash, 1, brews)
insert_into("sppl.PROD_BREWS_MASHINGS", 5, prod_brews_mash, cur, conn)

In [ ]:
num_brews_mash_rest = num_rows
prod_brews_mash_rest = generate_PROD_BREWS_MASHINGS_RESTS(num_brews_mash_rest, 1, brews)
insert_into("sppl.PROD_BREWS_MASHING_RESTS", 3, prod_brews_mash_rest, cur, conn)

In [ ]:
num_brews_spargings = num_rows
prod_brews_spargings = generate_PROD_BREWS_SPARGINGS(num_brews_spargings, 1, brews)
insert_into("sppl.PROD_BREWS_SPARGINGS", 3, prod_brews_spargings, cur, conn)

In [ ]:
num_brews_boilings = num_rows
prod_brews_boilings = generate_PROD_BREW_BOILINGS(num_brews_boilings, 1, brews)
insert_into("sppl.PROD_BREWS_BOILINGS", 3, prod_brews_boilings, cur, conn)

In [ ]:
"""
Table removed from DB
num_brews_hop = num_rows
prod_brews_hop = generate_PROD_BREWS_HOP_ADDINGS(num_brews_hop, 1, brews)
insert_into("sppl.PROD_BREWS_HOP_ADDINGS", 3, prod_brews_hop, cur, conn)
"""

In [ ]:
"""
Table removed from DB
num_brews_yeast = num_rows
prod_brews_yeast = generate_PROD_BREWS_YEAST_ADDINGS(num_brews_yeast, 1, brews)
insert_into("sppl.PROD_BREWS_YEAST_ADDINGS", 3, prod_brews_yeast, cur, conn)
"""

# Sales

In [ ]:
num_customers = num_rows
sales_customers, cols = generate_SALES_CUSTOMERS(num_customers)
insert_into("sppl.SALES_CUSTOMERS", 6, sales_customers, cur, conn, col = cols)
sales_customers = pd.read_sql_query("SELECT * FROM sppl.SALES_CUSTOMERS", conn)

In [ ]:
num_products = num_rows
sales_product, cols = generate_SALES_PRODUCT(num_products, rec)
insert_into("sppl.SALES_PRODUCT", 4, sales_product, cur, conn, col = cols)
sales_product = pd.read_sql_query("SELECT * FROM sppl.SALES_PRODUCT", conn)

In [ ]:
num_bottlings = num_rows
sales_product_db = pd.read_sql_query("SELECT * FROM sppl.SALES_BOTTLINGS", conn)
sales_bottlings, cols = generate_SALES_BOTTLINGS(num_bottlings, sales_product, sales_product_db)
insert_into("sppl.SALES_BOTTLINGS", 4, sales_bottlings, cur, conn, col = cols)
sales_product = pd.read_sql_query("SELECT * FROM sppl.SALES_BOTTLINGS", conn)

In [ ]:
num_stock_product = num_rows
sales_stock_products_db = pd.read_sql_query("SELECT * FROM sppl.SALES_STOCK_PRODUCT", conn)
sales_stock_products = generate_SALES_STOCK_PRODUCT(num_stock_product, sales_product, sales_stock_products_db)
insert_into("sppl.SALES_STOCK_PRODUCT", 3, sales_stock_products, cur, conn)

In [ ]:
num_orders = num_rows
sales_orders,cols = generate_SALES_ORDER(num_orders, sales_customers)
insert_into("sppl.SALES_ORDER", 3, sales_orders, cur, conn, col = cols)
sales_orders = pd.read_sql_query("SELECT * FROM sppl.SALES_ORDER", conn)

In [ ]:
num_order_items = num_orders*3
sales_order_items_db = pd.read_sql_query("SELECT * FROM sppl.SALES_ORDER_ITEM", conn)
sales_order_items = generate_SALES_ORDER_ITEM(num_orders, sales_orders, sales_product,sales_order_items_db)
insert_into("sppl.SALES_ORDER_ITEM", 3, sales_order_items, cur, conn)
sales_order_items = pd.read_sql_query("SELECT * FROM sppl.SALES_ORDER_ITEM", conn)

In [ ]:
cur.close()
conn.close()